# 1. Creating Recommendations

## 1. Load the Data

The first step in evaluating the smart home recommendation system is to create recommendations using the Recommendation Agent. These recommendations will be used in simulating usage patterns when the system is in operation. The Recommendation Agent gathers inputs from the Activity Agent, Usage Agent, Load Agent, and Price Agent, and uses this information to provide recommendations for all shiftable devices within a given household on a specific date.

Throughout the scripts, we rely on dictionaries to efficiently manage and organize the data necessary for the smart home recommendation system. By employing dictionaries, we store information using intuitive keys, such as household and device identifiers, and easily iterate through the data.

In [2]:
import pandas as pd
import numpy as np
import pickle

from helper_functions import Helper
from agents import Evaluation_Agent, Preparation_Agent, Activity_Agent
from agents import Activity_Agent, Usage_Agent, Load_Agent


In [3]:
import os
DATA_PATH = os.getcwd()+'/data/'
DATA_PATH


'/Users/iremzalp/Desktop/Thesis/Probabilistic_Impact_Estimation_RecSys/data/'

The recommendations focus on shiftable devices found within households. The usage of these devices can be rescheduled within a day. The 'devices' dictionary holds information about all available devices for testing the recommendation system. We will iterate through this dictionary using the recommendation acceptance algorithm. To tailor the script for generating recommendations exclusively for specific devices or households of interest, users can comment out the nonrelevant entries in this dictionary.

In [4]:
devices = {0: {'hh': 'hh1', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          1: {'hh': 'hh1', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          2: {'hh': 'hh2', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          3: {'hh': 'hh2', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          4: {'hh': 'hh3', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          5: {'hh': 'hh3', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          6: {'hh': 'hh3', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          7: {'hh': 'hh4', 'dev_name': 'Washing Machine (1)', 'dev': 'washing_machine'},
          8: {'hh': 'hh4', 'dev_name': 'Washing Machine (2)', 'dev': 'washing_machine'},
          9: {'hh': 'hh5', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          10: {'hh': 'hh6', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          11: {'hh': 'hh6', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          12: {'hh': 'hh7', 'dev_name': 'Tumble Dryer', 'dev': 'tumble_dryer'},
          13: {'hh': 'hh7', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          14: {'hh': 'hh7', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          15: {'hh': 'hh8', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          16: {'hh': 'hh9', 'dev_name': 'Washer Dryer', 'dev': 'washer_dryer'},
          17: {'hh': 'hh9', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          18: {'hh': 'hh9', 'dev_name': 'Dishwasher', 'dev': 'dishwasher'},
          19: {'hh': 'hh10', 'dev_name': 'Washing Machine', 'dev': 'washing_machine'},
          }



To streamline the data input process for our recommendation agents, we compile the 'shiftable_devices_list.' This list provides a summary of the shiftable devices available in each household. It, along with the household ID, is necessary for configuring the agents.

In [5]:
from helper_functions_thesis import Helper_Functions_Thesis
shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(devices)

shiftable_devices_dict


{'hh1': ['Washing Machine', 'Dishwasher'],
 'hh2': ['Washing Machine', 'Dishwasher'],
 'hh3': ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
 'hh4': ['Washing Machine (1)', 'Washing Machine (2)'],
 'hh5': ['Tumble Dryer'],
 'hh6': ['Washing Machine', 'Dishwasher'],
 'hh7': ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
 'hh8': ['Washing Machine'],
 'hh9': ['Washer Dryer', 'Washing Machine', 'Dishwasher'],
 'hh10': ['Washing Machine']}

We further require another dictionary known as the 'active_appliances_list.' This dictionary comprises devices that require user interaction. The Activity agent utilizes this list to predict household activities.

In [6]:
active_appliences_dict_complete = {
    "hh1" : ['Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Electric Heater'], #'Tumble Dryer', 
    "hh2" : ['Washing Machine', 'Dishwasher', 'Television', 'Microwave', 'Toaster', 'Hi-Fi', 'Kettle', 'Oven Extractor Fan'],
    "hh3" : ['Tumble Dryer', 'Washing Machine', 'Dishwasher','Toaster', 'Television', 'Microwave', 'Kettle'],
    "hh4" : ['Washing Machine (1)', 'Washing Machine (2)', 'Computer Site', 'Television Site', 'Microwave', 'Kettle'],
    "hh5" : ['Tumble Dryer', 'Computer Site', 'Television Site', 'Combination Microwave', 'Kettle', 'Toaster'], # , 'Washing Machine' --> consumes energy constantly; , 'Dishwasher' --> noise at 3am
    "hh6" : ['Washing Machine', 'Dishwasher', 'MJY Computer', 'Television Site', 'Microwave', 'Kettle', 'Toaster', 'PGM Computer'],
    "hh7" : ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Toaster', 'Kettle'],
    "hh8" : ['Washing Machine', 'Toaster', 'Computer', 'Television Site', 'Microwave', 'Kettle'], # 'Dryer' --> consumes constantly
    "hh9" : ['Washer Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Microwave', 'Kettle', 'Hi-Fi', 'Electric Heater'], 
    "hh10" : ['Washing Machine', 'Magimix (Blender)','Television Site', 'Microwave', ' Kenwood KMix'] #'Dishwasher'
}

active_appliences_dict = {key: active_appliences_dict_complete[key] for key in shiftable_devices_dict}
active_appliences_dict.keys()


dict_keys(['hh1', 'hh2', 'hh3', 'hh4', 'hh5', 'hh6', 'hh7', 'hh8', 'hh9', 'hh10'])

We set the parameters used by the agents. These are used by the Preparation Agent to prepare the data for other Agents. A detailed overview can be found in the Preparation Agent script.

In [7]:
threshold = 0.01
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher'] 
active_appliances = ['Toaster', 'Tumble Dryer', 'Dishwasher', 'Washing Machine','Television', 'Microwave', 'Kettle']

#load_parameters
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}

device_params = {
    'threshold': 0.15
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

#activity_parameters
activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold 
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#usage_parameters
device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}


We begin by generating outputs from the Usage Agent, Availability Agent, and Load Agent. The Recommendation Agent will then combine these outputs to generate recommendations. To expedite the process, pre-generated outputs for all households can be uploaded from the cell below.

In [9]:
load_dict = Helper_Functions_Thesis.open_pickle_file(DATA_PATH, 'load_dict.pickle')
activity_dict = Helper_Functions_Thesis.open_pickle_file(DATA_PATH, 'activity_dict.pickle')
usage_dict = Helper_Functions_Thesis.open_pickle_file(DATA_PATH, 'usage_dict.pickle')


In [10]:
from helper_functions import Helper
helper = Helper()
price_df = helper.create_day_ahead_prices_df(DATA_PATH,'Day-ahead Prices_201501010000-201601010000.csv')


##  2. Conduct Grid Search for Availability and Usage Thresholds

We aim to test the recommendation system over a 96-day period, starting from 01.02.2015. To use the recommendation agent, we need to define two hyperparameters: availability and usage thresholds. These thresholds govern when the recommendation system provides suggestions. Striking the right balance between these thresholds is essential. Setting them too low results in frequent recommendations, increasing costs, while overly high thresholds may not align with real-life use patterns.

To strike the right balance, we conduct a hyperparameter tuning process through grid search for each household. This process maximizes daily recommendation accuracy and allows us to capture device usage as realistically as possible while minimizing costs. 

We've established **'threshold_dict'** as a configuration tool, designed to fine-tune activity and usage thresholds within the recommendation system. This dictionary contains values that define the parameter space for a systematic grid search. We've chosen a granularity of 0.05 to control the level of detail in our exploration. This process guides us in selecting thresholds that align best with real-world usage patterns.

We determine the thresholds from the validation data, since using testing data for parameter determination can introduce bias and overfitting. We aim to test the recommendation system on new and unseen data. Therefore, we perform the selection based on the validation period, which covers the period from 01.01.2015 to 31.01.2015.

In [11]:
threshold_dict = {
    'activity' : {'start': 0.0, 'end': 0.99, 'granularity': 0.05},
    'usage' : {'start': 0.0, 'end': 0.99, 'granularity': 0.05}
        }

### 2.1 Initialize the Class 

We introduce the **'Activity_Usage_Threshold_Search'** class, which streamlines the process of determining optimal thresholds. We define the class and initialize it with the required inputs.


We choose the Random Forest model to calculate availability and usage predictions due to its consistent performance, stability, and ability to leverage additional weather data. This choice establishes a robust foundation for our recommendation system, as demonstrated in the paper "Explainable Multi-Agent Recommendation System for Energy-Efficient Decision Support in Smart Homes".



In [12]:
model_type='random forest'
recommendation_start = '2015-01-01'
recommendation_length = 31

In [13]:
from typing import Dict, Any
class Activity_Usage_Threshold_Search:
    '''
    A class for streamlining the process of determining optimal thresholds for activity and usage.
    Parameters:
    ----------
    activity_dict : 
        A dictionary containing the output dataframes from the Activity Agent for each household.
    usage_dict :
        A dictionary containing the output dataframes from the Usage Agent for each household.
    load_dict :
        A dictionary containing the output dataframes from the Load Agent for each household.
    devices :
        A dictionary containing device-specific information.
    price_df : 
        A DataFrame with hourly price data in GBP per megawatt-hour.
    recommendation_start :
        The start date of the recommendation validation period in 'yyyy-mm-dd' format.
    recommendation_length : 
        The length of the validation period in days.
    model_type : 
        The machine learning model type used for predicting availability and usage probabilities.
    threshold_dict : 
        A dictionary defining the availability and usage threshold parameter space.
    '''
    def __init__(
        self,
        activity_dict: Dict[str, pd.DataFrame],
        usage_dict: Dict[str, pd.DataFrame],
        load_dict: Dict[str, pd.DataFrame],
        devices: Dict[str, Any],
        price_df: pd.DataFrame,
        recommendation_start: str,
        recommendation_length: int,
        model_type: str = "random forest",
        threshold_dict: Dict[str, Any] = {
            "activity": {"start": 0.0, "end": 0.95, "granularity": 0.05},
            "usage": {"start": 0.0, "end": 0.95, "granularity": 0.05},
        },
    ):
        import pandas as pd
        from helper_functions_thesis import Helper_Functions_Thesis
        self.activity_dict = {dev: df.copy() for dev, df in activity_dict.items()}
        self.usage_dict = {dev: df.copy() for dev, df in usage_dict.items()}
        self.load_dict = {dev: df.copy() for dev, df in load_dict.items()}
        self.price_df = price_df
        self.devices = devices
        self.model_type = model_type
        self.threshold_dict = threshold_dict
        
        self.date_list = Helper_Functions_Thesis.create_date_list_daily(
            recommendation_start, recommendation_length
        )
        
        self.shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(
            self.devices
        )


In [14]:
threshold_search = Activity_Usage_Threshold_Search(
    activity_dict = activity_dict,
    usage_dict = usage_dict,
    load_dict = load_dict,
    devices = devices,
    price_df = price_df,
    recommendation_start = recommendation_start,
    recommendation_length = recommendation_length,
    model_type = model_type,
    threshold_dict = threshold_dict,
)


### 2.2 Initialize the Recommendation Agent

We proceed by initializing the Recommendation Agent using the preprocessed data and the list of the shiftable device names for which we intend to make predictions.

In [15]:
def initialize_recommendation_agent(self):
    from agents import X_Recommendation_Agent
    recommendation_agent_dict = {
        hh: X_Recommendation_Agent(
            self.activity_dict.get(hh),
            self.usage_dict.get(hh),
            self.load_dict.get(hh),
            self.price_df,
            self.shiftable_devices_dict.get(hh),
            model_type=self.model_type
        )
        for hh in self.shiftable_devices_dict.keys()
    }
    return recommendation_agent_dict

setattr(Activity_Usage_Threshold_Search, 'initialize_recommendation_agent', initialize_recommendation_agent)
del initialize_recommendation_agent


In [16]:
recommendation_agent_dict = threshold_search.initialize_recommendation_agent()


### 2.3 Generate Daily Recommendations Across Prediction Period for Individual Households


The Recommendation Agent features a **'pipeline'** function responsible for generating recommendations for each shiftable device within a household. Given that our testing phase spans for several months, we build the **'create_recommendations'** method. This method iterates through the days within the prediction range and generates daily recommendations for a given household. It returns a DataFrame containing the recommended launch hours for all shiftable devices in the household.

If the calculated activity or usage probability falls below the specified threshold, the system suggests no recommendation, and the function returns NaN.

The **'create_recommendations'** method is placed in the **'Helper_Functions_Thesis'** class rather than the **'Activity_Usage_Threshold_Search'** class to facilitate its reuse by multiple classes throughout the codebase.

In [17]:
def create_recommendations(date_list, hh, recommendation_agent_dict, activity_prob_threshold=0.5, usage_prob_threshold=0.6):
    import pandas as pd
    
    recommendations_table = pd.DataFrame(columns=["recommendation_date", "device", "recommendation"])

    for date in date_list:
        print(date)
        recommendation_df = recommendation_agent_dict[hh].pipeline(
            date=str(date),
            activity_prob_threshold=activity_prob_threshold,
            usage_prob_threshold=usage_prob_threshold,
            evaluation=False,
            weather_sel=True
        )[
            ["recommendation_date", "device", "recommendation"]
        ]

        recommendations_table = pd.concat([recommendations_table, recommendation_df])

        recommendations_table.recommendation = pd.to_numeric(recommendations_table.recommendation,
                                                                        downcast='integer',
                                                                        errors='coerce')
    
    return recommendations_table


### 2.4 Perform Grid Search

To start the grid search process, we begin by defining candidate thresholds. The **'generate_probability_list'** and **'create_threshold_values'** methods are designed to create a list of probability values based on the input configuration (start, end, and granularity of the probability range) provided in the **'threshold_dict'** dictionary. 

In [18]:
def generate_probability_list(self, params):
    import numpy as np
    
    start = params['start']
    end = params['end']
    granularity = params['granularity']
    
    prob_list = [round(prob, 2) for prob in np.arange(start, end, granularity)]
    
    return prob_list

setattr(Activity_Usage_Threshold_Search, 'generate_probability_list', generate_probability_list)
del generate_probability_list


In [19]:
def create_threshold_values(self):
    usage_params = self.threshold_dict['usage']
    activity_params = self.threshold_dict['activity']
    
    usage_prob_list = self.generate_probability_list(usage_params)
    activity_prob_list = self.generate_probability_list(activity_params)

    return usage_prob_list, activity_prob_list

setattr(Activity_Usage_Threshold_Search, 'create_threshold_values', create_threshold_values)
del create_threshold_values


In [20]:
usage_prob_list, activity_prob_list = threshold_search.create_threshold_values()
print(activity_prob_list)

[0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


The generated probability threshold lists are employed in the **"grid_search_probabiliy_thresholds"** method to systematically explore the combinations of these thresholds. For each household and for each pair of activity and usage probability thresholds, the function generates daily recommendations using the **"create_recommendations"** function. The results are stored in a dictionary for further analysis and comparison.

In [21]:
def grid_search_probability_thresholds(self, usage_prob_list, activity_prob_list, recommendation_agent_dict):
    from helper_functions_thesis import Helper_Functions_Thesis
    
    recommendations_dict = {}
    
    for hh in recommendation_agent_dict.keys():
        recommendations_dict[hh] = {}

        for i in activity_prob_list:
            for j in usage_prob_list:
                print(hh,i,j)
                recommendations_dict[hh][f"{i}_{j}"] = Helper_Functions_Thesis.create_recommendations(
                    date_list = self.date_list,
                    hh = hh,
                    recommendation_agent_dict = recommendation_agent_dict,
                    activity_prob_threshold = i,
                    usage_prob_threshold = j
                )

    return recommendations_dict

setattr(Activity_Usage_Threshold_Search, 'grid_search_probability_thresholds', grid_search_probability_thresholds)
del grid_search_probability_thresholds

Given that the exhaustive iteration through all combinations is time-intensive, we provide you with the pre-generated dictionary of recommendations for each household across the prediction period.

In [25]:
recommendations_dict = Helper_Functions_Thesis.open_pickle_file(DATA_PATH, 'recommendations_dict.pickle')

### 2.5 Create Accuracy Grid

Subsequently, we utilize the **'recommendations_dict'**, output of the **'grid_search_probabiliy_thresholds'** method to compare the true device usage patterns with the output of the recommendation system. The **'create_accuracy_grid'** method evaluates the alignment between the recommendation system's device usage predictions and the true device usage data across different scenarios of activity and usage probabilities. This is achieved by counting the days when the predicted and true daily usage don't align, giving us a complete view of the system's accuracy. By finding the combination that minimizes the total errors across days, we can assess the most accurate recommendation setup for the household.

In [26]:
def create_accuracy_grid(self, usage_prob_list, activity_prob_list, recommendations_dict):
    accuracy_grid = {}

    for hh, device in self.shiftable_devices_dict.items():
        accuracy_grid[hh] = pd.DataFrame(index=activity_prob_list, columns=usage_prob_list)
        accuracy_grid[hh].index.name = 'Usage Probabilities'
        accuracy_grid[hh].columns.name = 'Activity Probabilities'

        for a in activity_prob_list:
            for u in usage_prob_list:
                cols = [x + '_usage' for x in device]
                
                # Transform recommendaiton data to binary, daily usage patterns
                recommended_usage_pattern = recommendations_dict[hh][f"{a}_{u}"].copy()
                recommended_usage_pattern['recommendation'] = recommended_usage_pattern['recommendation'].apply(
                    lambda x: 1 if pd.notnull(x) else 0
                )

                recommended_usage_pattern = recommended_usage_pattern.set_index('recommendation_date')
                recommended_usage_pattern = (
                    recommended_usage_pattern.groupby([recommended_usage_pattern.index, 'device'])
                    ['recommendation']
                    .aggregate('first')
                    .unstack()[device]
                )
                
                recommended_usage_pattern.columns.name = None
                
                # Retrieve true device usage data
                true_usage_pattern = self.usage_dict[hh][cols].loc[self.date_list]
                true_usage_pattern.columns = device
                true_usage_pattern.index = recommended_usage_pattern.index
                
                # Compare recommended vs. true usage patterns and calculate differences
                usage_comparison = recommended_usage_pattern.compare(true_usage_pattern)

                if usage_comparison.empty:
                    accuracy_grid[hh].at[u, a] = 0
                else:
                    difference = usage_comparison.xs('self', axis=1, level=1, drop_level=False).count().sum()
                    accuracy_grid[hh].at[u, a] = difference 

        accuracy_grid[hh] = accuracy_grid[hh].astype(int)

    return accuracy_grid

setattr(Activity_Usage_Threshold_Search, 'create_accuracy_grid', create_accuracy_grid)
del create_accuracy_grid


In [27]:
accuracy_grid = threshold_search.create_accuracy_grid(usage_prob_list, activity_prob_list, recommendations_dict)

In [28]:
accuracy_grid['hh10']

Activity Probabilities,0.00,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95
Usage Probabilities,,,,,,,,,,,,,,,,,,,,
0.00,10,10,10,10,10,10,10,10,10,10,10,10,10,11,11,10,10,11,11,10
0.05,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,9,10,8
0.10,8,8,7,8,8,7,8,7,7,7,7,8,8,7,8,8,8,8,8,8
0.15,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,8,8,7
0.20,7,7,7,7,7,7,7,7,6,6,7,7,7,7,6,7,6,7,7,7
0.25,6,6,6,6,6,6,6,6,7,6,6,6,6,6,6,6,6,7,7,7
0.30,6,5,5,5,5,5,5,5,5,6,5,5,6,5,5,5,5,6,6,6
0.35,4,5,4,4,4,5,4,5,4,4,4,5,5,4,4,5,5,4,5,5
0.40,3,2,3,3,3,2,4,2,3,3,2,2,3,3,3,3,3,3,4,4


### 2.6 Return Best Threshold Combination

Finally, with the **'return_best_recommendations'** method, we identify the best thresholds and create a dictionary containing the optimal values for each household.

In [29]:
def return_best_recommendations(self, accuracy_grid, daily_recommendations_dict):
    threshold_dict = {}

    for hh in self.shiftable_devices_dict.keys():
        min_value = float("inf")
        min_index = None

        for col in accuracy_grid[hh].columns:
            for index, value in list(enumerate(accuracy_grid[hh][col])):
                if value < min_value:
                    min_value = value
                    min_index = (index, col)

        usage_threshold = accuracy_grid[hh].index[min_index[0]]
        activity_threshold = min_index[1]

        threshold_dict[hh] = {
            'usage': usage_threshold,
            'activity': activity_threshold
        }

    return threshold_dict

setattr(Activity_Usage_Threshold_Search, 'return_best_recommendations', return_best_recommendations)
del return_best_recommendations


In [30]:
best_thresholds = threshold_search.return_best_recommendations(accuracy_grid, recommendations_dict)
best_thresholds

{'hh1': {'usage': 0.65, 'activity': 0.05},
 'hh2': {'usage': 0.5, 'activity': 0.05},
 'hh3': {'usage': 0.7, 'activity': 0.0},
 'hh4': {'usage': 0.35, 'activity': 0.0},
 'hh5': {'usage': 0.3, 'activity': 0.0},
 'hh6': {'usage': 0.5, 'activity': 0.1},
 'hh7': {'usage': 0.55, 'activity': 0.0},
 'hh8': {'usage': 0.5, 'activity': 0.05},
 'hh9': {'usage': 0.5, 'activity': 0.05},
 'hh10': {'usage': 0.55, 'activity': 0.0}}

### 2.7 Pipeline

In [31]:
def pipeline(self):
    
    recommendation_agent_dict = self.initialize_recommendation_agent()
    
    usage_prob_list, activity_prob_list = self.create_threshold_values()
        
    daily_recommendations_dict = self.grid_search_probability_thresholds(usage_prob_list, activity_prob_list, recommendation_agent_dict)
        
    accuracy_grid = self.create_accuracy_grid(usage_prob_list, activity_prob_list, daily_recommendations_dict)
        
    best_thresholds = self.return_best_recommendations(accuracy_grid, daily_recommendations_dict)
        
    return best_thresholds

setattr(Activity_Usage_Threshold_Search, 'pipeline', pipeline)
del pipeline


## 3. Simulating Recommendation Acceptance

Having identified the optimal usage and activity threshold combination on the validation data, we are ready to create recommendations for the testing phase and build the simulated treatment data.  

### 3.1 Initialize the Class

We introduce the **'Create_Accept_Recommendations**' class, which simplifies the process of generating recommendations for the test phase using optimal thresholds and simulating device usage patterns influenced by the recommendation system. First we initialize the class with the necessary input parameters.

In [33]:
from typing import Dict, Any, Tuple
class Create_Accept_Recommendations:
    '''
    A class for generating recommendations and simulating device usage patterns through the test phase
    Parameters:
    ----------
    recommendation_agent_dict : 
        A dictionary containing household recommendation agents to create recommendations.
    devices :
        A dictionary containing device-specific information.
    recommendation_start :
        The start date of the recommendation validation period in 'yyyy-mm-dd' format.
    recommendation_length : 
        The length of the recommendation validation period in days.
    best_thresholds : 
        A dictionary defining the best availability and usage thresholds for each household.
    load_dict :
        A dictionary containing the output dataframes from the Load Agent for each household.
    '''
    def __init__(
        self,
        recommendation_agent_dict,
        devices: Dict[str, Any],
        recommendation_start: str,
        recommendation_length: int,
        best_thresholds,
        load_dict: Dict[str, pd.DataFrame],
    ):
        import pandas as pd
        from helper_functions_thesis import Helper_Functions_Thesis
        self.recommendation_agent_dict = recommendation_agent_dict
        self.devices = devices
        self.best_thresholds = best_thresholds
        self.recommendation_start = recommendation_start
        self.recommendation_length = recommendation_length
        self.shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(devices)
        self.date_list = Helper_Functions_Thesis.create_date_list_daily(
            recommendation_start, recommendation_length
        )
        self.load_dict = {dev: df.copy() for dev, df in load_dict.items()}


In [34]:
create_accept_recommendations = Create_Accept_Recommendations(
    recommendation_agent_dict = recommendation_agent_dict,
    devices = devices,
    best_thresholds = best_thresholds,
    recommendation_start = '2015-02-01',
    recommendation_length = 96,
    load_dict = load_dict
)

### 3.2 Generate Daily Recommendations with Chosen Thresholds

The **'generate_daily_recommendations'** method employs the **'create_recommendations'** function and creates a dictionary of daily recommendations over the testing period for each household.

In [35]:
def generate_daily_recommendations(self):
    from agents import X_Recommendation_Agent
    from helper_functions_thesis import Helper_Functions_Thesis
    
    best_daily_recommendations_dict = {}

    for hh in self.shiftable_devices_dict.keys():
        activity_threshold = self.best_thresholds[hh]['activity']
        usage_threshold = self.best_thresholds[hh]['usage']
        print(f"Household: {hh}, Activity Threshold: {activity_threshold}, Usage Threshold: {usage_threshold}")

        best_daily_recommendations_dict[hh] = Helper_Functions_Thesis.create_recommendations(
            date_list = self.date_list,
            hh = hh,
            recommendation_agent_dict = self.recommendation_agent_dict,
            activity_prob_threshold = activity_threshold,
            usage_prob_threshold = usage_threshold
        )
    
    return best_daily_recommendations_dict

setattr(Create_Accept_Recommendations, 'generate_daily_recommendations', generate_daily_recommendations)
del generate_daily_recommendations


In [38]:
#recommendations_dict_test = create_accept_recommendations.generate_daily_recommendations()
recommendations_dict_test = Helper_Functions_Thesis.open_pickle_file(DATA_PATH, 'recommendations_dict_test.pickle')

### 3.3 Preparing Data for the Recommendation Acceptance Algorithm

Having generated the recommendations, our attention now turns to defining the recommendation acceptance function and constructing a realistic alternative scenario. An important preliminary step in this process involves integrating device-specific recommendations into the devices dictionary using the **'update_device_information'** method. This adjustment enables us to conduct the recommendation acceptance process on a per-device basis. Furthermore, we compute the average load profile for each device using the Load Agent. These profiles are essential in generating usages on days when the recommendation system provides suggestions despite the absence of usage.

In [39]:
def update_device_information(self, recommendations_dict):
    from agents import Load_Agent
    
    for dev, device_information in self.devices.items():
        hh = device_information['hh']
        dev_name = device_information['dev_name']

        load_agent = Load_Agent(hh[2:])
        load_profiles = load_agent.pipeline(self.load_dict[hh], self.recommendation_start, self.shiftable_devices_dict[hh]) 

        device_information['usage'] = self.load_dict[hh][dev_name]
        device_information['load_profile'] = load_profiles.loc[dev_name]
        device_information['recommendation'] = recommendations_dict[hh][recommendations_dict[hh]['device'] == dev_name]

setattr(Create_Accept_Recommendations, 'update_device_information', update_device_information)
del update_device_information


In [40]:
create_accept_recommendations.update_device_information(recommendations_dict_test)

### 3.4 Recommendation Acceptance Algorithm

Now we can simulate the acceptance of the recommendations. 
Firt we create two helper functions. First one, the *'calculate_average_usage_length'* method calculates the average length of a device usage, and the second one, the *'_insert_usage_values_from_load_profile'* method inserts electricity consumption values from the device specific load profiles. 
 

In [41]:
@staticmethod
def _calculate_average_usage_length(device_usage):
    # Add zero padding at both ends for edge detection
    device_usage = device_usage.fillna(0).copy()
    device_usage[device_usage != 0] = 1
    padded_usage = np.concatenate(([0], device_usage, [0]))

    # Detect changes in device usage
    change_indices = np.flatnonzero(padded_usage[1:] != padded_usage[:-1])

    # Calculate the durations of usage periods
    period_lengths = change_indices[1:] - change_indices[:-1]

    average_usage_length = period_lengths[::2].mean()
    average_usage_length = min(22,np.floor(average_usage_length))

    return int(average_usage_length), average_usage_length % 1

setattr(Create_Accept_Recommendations, '_calculate_average_usage_length', _calculate_average_usage_length)
del _calculate_average_usage_length

In [42]:
def _insert_usage_values_from_load_profile(
    self,
    daily_usage_extended,
    recommended_start_hour,
    load_profile,
):
        
    average_usage_int, average_usage_dec = self._calculate_average_usage_length(load_profile)
    daily_usage_extended[recommended_start_hour:recommended_start_hour+average_usage_int] = load_profile[0:average_usage_int]
    daily_usage_extended[recommended_start_hour+average_usage_int] = load_profile[average_usage_int+1]*average_usage_dec
    daily_usage_slice = daily_usage_extended[recommended_start_hour:recommended_start_hour+average_usage_int+1]

    return daily_usage_slice

setattr(Create_Accept_Recommendations, '_insert_usage_values_from_load_profile', _insert_usage_values_from_load_profile)
del _insert_usage_values_from_load_profile

The **'accept_recommendations'** method takes as input two binary parameters. The *'is_usage_added'* parameter plays a pivotal role in shaping the recommendation acceptance process, allowing us to differentiate between two distinct scenarios. When set to False, the system aligns with realistic usage patterns, accepting recommendations exclusively on days with prior device usage. In contrast, setting it to True signifies full recommendation acceptance on all days, including days without historical device usage. In this scenario, the system introduces the device's average load profile to days without usage. While the former scenario doesnt allow additional consumption and portrays the complete cost-saving potential, the latter displays a more realistic setting, given that we set the acceptance rate to 100%. The *'is_info_displayed'* parameter controls whether informational messages are printed during the execution of the recommendation acceptance function, offering insights into the process when set to True.

This function operates iteratively across each day in the specified date list. For days with available recommendations, the function shifts historical usage data to match the recommended hour. Alternatively, if there is no recorded usage for that day and the *'is_usage_added'* parameter is set to True, the function inserts the device's average load profile to mimic device usage. In cases where no recommendations are present, the function proceeds to the subsequent date in the list. By repeating this process for all dates in the "date_list," the function yields simulated energy consumption data reflecting the outcome of the recommendations.

In [43]:
def accept_recommendations(self, is_usage_added=False, is_info_displayed=False):
    from datetime import timedelta
    import numpy as np
    import pandas as pd
    
    load_post_recommendation = {}
    
    # Iterate over each device
    for dev in self.devices.keys():
        device_usage = self.devices[dev]['usage'].copy()
        load_profile = self.devices[dev]['load_profile'].copy()
        recommendation = self.devices[dev]['recommendation'].copy()
    
        device_usage = device_usage[str(self.date_list[0]):str(self.date_list[-1])]
        output = device_usage.copy()

        # Iterate over each date in the specified date list
        for date in self.date_list:

            date_before = date - timedelta(days=1)
            date = str(date)

            daily_usage = device_usage.loc[date].copy()
            daily_usage_extended = device_usage.loc[date:][:48].copy()
            
            # Retrieve the recommended start hour
            recommended_start_hour = recommendation[(recommendation.recommendation_date == date)]['recommendation'].iloc[0]
            
            if np.isnan(recommended_start_hour):
                print(date, "No recommendation") if is_info_displayed else None
                continue

            recommended_start_hour = int(recommended_start_hour)
            
            # Extract the usage start hours
            shifted_daily_usage = daily_usage.shift()

            if date != str(self.date_list[0]):
                shifted_daily_usage.iloc[0] = device_usage.loc[str(date_before)].iloc[-1]

            start_hours_usage = daily_usage[
                (daily_usage.values > 0) & (shifted_daily_usage.values == 0)
            ].index.hour.tolist()
            
            if start_hours_usage == []:
                if is_usage_added:
                    #If there is no usage on recommendation day, add usage data from load profile
                    daily_usage_slice = self._insert_usage_values_from_load_profile(daily_usage_extended,
                                                                                    recommended_start_hour,
                                                                                    load_profile,)
                    output = daily_usage_slice.combine_first(output)
                    continue
                else:
                    print(date, "No usage") if is_info_displayed else None
                    continue

            # Determine the closest start hour to the recommended hour
            distance_of_closest_start = int(min(np.full(shape=len(start_hours_usage), fill_value=recommended_start_hour) - start_hours_usage, key=abs, default=0))
            shifted_usage_start = int(recommended_start_hour - distance_of_closest_start)
            
            if (len(start_hours_usage) != 0) and (shifted_usage_start == recommended_start_hour):
                print(date, "Device usage starts at the recommended hour") if is_info_displayed else None
                continue

            # Calculate the length of device usage
            usage_length = (
                ((daily_usage_extended.iloc[shifted_usage_start:] == 0).idxmax() - daily_usage_extended.index[shifted_usage_start])
                    // pd.Timedelta(minutes=60))

            print(date, "Shift load starting at", shifted_usage_start, "to the recommended hour", recommended_start_hour) if is_info_displayed else None

            usage_section = daily_usage_extended.iloc[shifted_usage_start:shifted_usage_start + usage_length]
            usage_shifted = usage_section.shift(periods=distance_of_closest_start, freq="60T")
            output[date:].iloc[shifted_usage_start:shifted_usage_start + usage_length] = 0
            output = usage_shifted.combine_first(output)

        # Store the results for the current device
        output = pd.DataFrame({'usage': output})
        load_post_recommendation[dev] = output
    
    return load_post_recommendation

setattr(Create_Accept_Recommendations, 'accept_recommendations', accept_recommendations)
del accept_recommendations

In [44]:
load_post_recommendation = create_accept_recommendations.accept_recommendations(is_usage_added=False, is_info_displayed=True)
load_post_recommendation_usage_added  = create_accept_recommendations.accept_recommendations(is_usage_added = True, is_info_displayed=False)

2015-02-01 Shift load starting at 5 to the recommended hour 3
2015-02-02 No recommendation
2015-02-03 No recommendation
2015-02-04 No recommendation
2015-02-05 No recommendation
2015-02-06 Shift load starting at 9 to the recommended hour 2
2015-02-07 No recommendation
2015-02-08 No recommendation
2015-02-09 No recommendation
2015-02-10 No recommendation
2015-02-11 No recommendation
2015-02-12 No recommendation
2015-02-13 Shift load starting at 4 to the recommended hour 3
2015-02-14 No usage
2015-02-15 Device usage starts at the recommended hour
2015-02-16 No usage
2015-02-17 No recommendation
2015-02-18 Shift load starting at 10 to the recommended hour 3
2015-02-19 No usage
2015-02-20 No recommendation
2015-02-21 Device usage starts at the recommended hour
2015-02-22 No recommendation
2015-02-23 Shift load starting at 11 to the recommended hour 4
2015-02-24 No recommendation
2015-02-25 Shift load starting at 12 to the recommended hour 4
2015-02-26 No recommendation
2015-02-27 No recomm

2015-04-22 Shift load starting at 8 to the recommended hour 2
2015-04-23 No recommendation
2015-04-24 No recommendation
2015-04-25 No recommendation
2015-04-26 Shift load starting at 8 to the recommended hour 3
2015-04-27 Shift load starting at 12 to the recommended hour 23
2015-04-28 No usage
2015-04-29 No recommendation
2015-04-30 No recommendation
2015-05-01 Shift load starting at 10 to the recommended hour 4
2015-05-02 Shift load starting at 0 to the recommended hour 3
2015-05-03 No recommendation
2015-05-04 Shift load starting at 11 to the recommended hour 23
2015-05-05 No recommendation
2015-05-06 No recommendation
2015-05-07 Shift load starting at 7 to the recommended hour 6
2015-02-01 No recommendation
2015-02-02 Shift load starting at 14 to the recommended hour 3
2015-02-03 No recommendation
2015-02-04 No recommendation
2015-02-05 Shift load starting at 14 to the recommended hour 23
2015-02-06 No recommendation
2015-02-07 No recommendation
2015-02-08 No usage
2015-02-09 No rec

2015-05-02 Shift load starting at 7 to the recommended hour 23
2015-05-03 No recommendation
2015-05-04 No recommendation
2015-05-05 No usage
2015-05-06 No recommendation
2015-05-07 No recommendation
2015-02-01 No recommendation
2015-02-02 Shift load starting at 12 to the recommended hour 3
2015-02-03 No recommendation
2015-02-04 No recommendation
2015-02-05 No recommendation
2015-02-06 No recommendation
2015-02-07 No recommendation
2015-02-08 Shift load starting at 14 to the recommended hour 23
2015-02-09 No recommendation
2015-02-10 No recommendation
2015-02-11 No recommendation
2015-02-12 No usage
2015-02-13 No recommendation
2015-02-14 No recommendation
2015-02-15 No recommendation
2015-02-16 No recommendation
2015-02-17 No recommendation
2015-02-18 No recommendation
2015-02-19 No recommendation
2015-02-20 No recommendation
2015-02-21 No recommendation
2015-02-22 No recommendation
2015-02-23 No recommendation
2015-02-24 No recommendation
2015-02-25 No recommendation
2015-02-26 No re

2015-03-24 Shift load starting at 20 to the recommended hour 3
2015-03-25 No recommendation
2015-03-26 Shift load starting at 7 to the recommended hour 3
2015-03-27 No recommendation
2015-03-28 Shift load starting at 12 to the recommended hour 4
2015-03-29 Shift load starting at 7 to the recommended hour 3
2015-03-30 Shift load starting at 20 to the recommended hour 23
2015-03-31 No usage
2015-04-01 No recommendation
2015-04-02 Shift load starting at 8 to the recommended hour 2
2015-04-03 No recommendation
2015-04-04 No recommendation
2015-04-05 No recommendation
2015-04-06 No recommendation
2015-04-07 No recommendation
2015-04-08 Shift load starting at 7 to the recommended hour 2
2015-04-09 Shift load starting at 20 to the recommended hour 2
2015-04-10 Shift load starting at 20 to the recommended hour 23
2015-04-11 Shift load starting at 19 to the recommended hour 3
2015-04-12 Shift load starting at 7 to the recommended hour 3
2015-04-13 Shift load starting at 14 to the recommended ho

2015-05-05 Shift load starting at 21 to the recommended hour 0
2015-05-06 No usage
2015-05-07 Shift load starting at 6 to the recommended hour 0
2015-02-01 No recommendation
2015-02-02 No recommendation
2015-02-03 No recommendation
2015-02-04 No recommendation
2015-02-05 No recommendation
2015-02-06 No recommendation
2015-02-07 No recommendation
2015-02-08 Shift load starting at 20 to the recommended hour 4
2015-02-09 Shift load starting at 9 to the recommended hour 2
2015-02-10 No usage
2015-02-11 No usage
2015-02-12 No recommendation
2015-02-13 No recommendation
2015-02-14 No recommendation
2015-02-15 Shift load starting at 12 to the recommended hour 4
2015-02-16 No recommendation
2015-02-17 No recommendation
2015-02-18 No recommendation
2015-02-19 No recommendation
2015-02-20 No recommendation
2015-02-21 No recommendation
2015-02-22 Shift load starting at 8 to the recommended hour 23
2015-02-23 No recommendation
2015-02-24 No recommendation
2015-02-25 No recommendation
2015-02-26 No

2015-04-03 No recommendation
2015-04-04 No recommendation
2015-04-05 No recommendation
2015-04-06 No recommendation
2015-04-07 No recommendation
2015-04-08 No recommendation
2015-04-09 No recommendation
2015-04-10 Shift load starting at 15 to the recommended hour 23
2015-04-11 Shift load starting at 10 to the recommended hour 3
2015-04-12 Shift load starting at 17 to the recommended hour 3
2015-04-13 Shift load starting at 7 to the recommended hour 0
2015-04-14 Shift load starting at 11 to the recommended hour 1
2015-04-15 Shift load starting at 20 to the recommended hour 2
2015-04-16 Shift load starting at 19 to the recommended hour 2
2015-04-17 Shift load starting at 11 to the recommended hour 2
2015-04-18 Shift load starting at 13 to the recommended hour 2
2015-04-19 Shift load starting at 19 to the recommended hour 3
2015-04-20 No usage
2015-04-21 Shift load starting at 9 to the recommended hour 23
2015-04-22 Shift load starting at 17 to the recommended hour 2
2015-04-23 Shift load

### 3.5 Pipeline

In [48]:
def pipeline(self):
    
    best_recommendations_dict = self.generate_daily_recommendations()

    self.update_device_information(best_recommendations_dict)
    
    load_post_recommendation = self.accept_recommendations(is_usage_added=False, is_info_displayed=False)
    
    load_post_recommendation_usage_added = self.accept_recommendations(is_usage_added=True, is_info_displayed=False)
    
    return load_post_recommendation, load_post_recommendation_usage_added


setattr(Create_Accept_Recommendations, 'pipeline', pipeline)
del pipeline

<br>
<br>
<br>

## **Appendix A1: Complete Activity_Usage_Threshold_Search Class**

In [38]:

class Activity_Usage_Threshold_Search:
    '''
    A class for streamlining the process of determining optimal thresholds for activity and usage.
    Parameters:
    ----------
    activity_dict : 
        A dictionary containing the output dataframes from the Activity Agent for each household.
    usage_dict :
        A dictionary containing the output dataframes from the Usage Agent for each household.
    load_dict :
        A dictionary containing the output dataframes from the Load Agent for each household.
    devices :
        A dictionary containing device-specific information.
    price_df : 
        A DataFrame with hourly price data in GBP per megawatt-hour.
    recommendation_start :
        The start date of the recommendation validation period in 'yyyy-mm-dd' format.
    recommendation_length : 
        The length of the validation period in days.
    model_type : 
        The machine learning model type used for predicting availability and usage probabilities.
    threshold_dict : 
        A dictionary defining the availability and usage threshold parameter space.
    '''
    def __init__(
        self,
        activity_dict: Dict[str, pd.DataFrame],
        usage_dict: Dict[str, pd.DataFrame],
        load_dict: Dict[str, pd.DataFrame],
        devices: Dict[str, Any],
        price_df: pd.DataFrame,
        recommendation_start: str,
        recommendation_length: int,
        model_type: str = "random forest",
        threshold_dict: Dict[str, Any] = {
            "activity": {"start": 0.0, "end": 0.95, "granularity": 0.05},
            "usage": {"start": 0.0, "end": 0.95, "granularity": 0.05},
        },
    ):
        import pandas as pd
        from helper_functions_thesis import Helper_Functions_Thesis
        self.activity_dict = {dev: df.copy() for dev, df in activity_dict.items()}
        self.usage_dict = {dev: df.copy() for dev, df in usage_dict.items()}
        self.load_dict = {dev: df.copy() for dev, df in load_dict.items()}
        self.price_df = price_df
        self.devices = devices
        self.model_type = model_type
        self.threshold_dict = threshold_dict
        
        self.date_list = Helper_Functions_Thesis.create_date_list_daily(
            recommendation_start, recommendation_length
        )
        
        self.shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(
            self.devices
        )

    def initialize_recommendation_agent(self):
        from agents import X_Recommendation_Agent
        recommendation_agent_dict = {
            hh: X_Recommendation_Agent(
                self.activity_dict.get(hh),
                self.usage_dict.get(hh),
                self.load_dict.get(hh),
                self.price_df,
                self.shiftable_devices_dict.get(hh),
                model_type=self.model_type
            )
            for hh in self.shiftable_devices_dict.keys()
        }
        return recommendation_agent_dict

    def generate_probability_list(self, params):
        import numpy as np
        start = params['start']
        end = params['end']
        granularity = params['granularity']
        
        prob_list = [round(prob, 2) for prob in np.arange(start, end, granularity)]
        
        return prob_list

    def create_threshold_values(self):
        usage_params = self.threshold_dict['usage']
        activity_params = self.threshold_dict['activity']
        
        usage_prob_list = self.generate_probability_list(usage_params)
        activity_prob_list = self.generate_probability_list(activity_params)

        return usage_prob_list, activity_prob_list

    def grid_search_probability_thresholds(self, usage_prob_list, activity_prob_list, recommendation_agent_dict):
        from helper_functions_thesis import Helper_Functions_Thesis
        recommendations_dict = {}
        
        for hh in recommendation_agent_dict.keys():
            recommendations_dict[hh] = {}

            for i in activity_prob_list:
                for j in usage_prob_list:
                    print(hh,i,j)
                    recommendations_dict[hh][f"{i}_{j}"] = Helper_Functions_Thesis.create_recommendations(
                        date_list=self.date_list,
                        hh=hh,
                        recommendation_agent_dict=recommendation_agent_dict,
                        activity_prob_threshold=i,
                        usage_prob_threshold=j
                    )

        return recommendations_dict

    def create_accuracy_grid(self, usage_prob_list, activity_prob_list, recommendations_dict):
        accuracy_grid = {}

        for hh, device in self.shiftable_devices_dict.items():
            accuracy_grid[hh] = pd.DataFrame(index=activity_prob_list, columns=usage_prob_list)
            accuracy_grid[hh].index.name = 'Usage Probabilities'
            accuracy_grid[hh].columns.name = 'Activity Probabilities'

            for a in activity_prob_list:
                for u in usage_prob_list:
                    cols = [x + '_usage' for x in device]
                    
                    # Transform recommendaiton data to binary, daily usage patterns
                    recommended_usage_pattern = recommendations_dict[hh][f"{a}_{u}"].copy()
                    recommended_usage_pattern['recommendation'] = recommended_usage_pattern['recommendation'].apply(
                        lambda x: 1 if pd.notnull(x) else 0
                    )

                    recommended_usage_pattern = recommended_usage_pattern.set_index('recommendation_date')
                    recommended_usage_pattern = (
                        recommended_usage_pattern.groupby([recommended_usage_pattern.index, 'device'])
                        ['recommendation']
                        .aggregate('first')
                        .unstack()[device]
                    )
                    
                    recommended_usage_pattern.columns.name = None
                    
                    # Retrieve true device usage data
                    true_usage_pattern = self.usage_dict[hh][cols].loc[self.date_list]
                    true_usage_pattern.columns = device
                    true_usage_pattern.index = recommended_usage_pattern.index

                    # Compare recommended vs. true usage patterns and calculate differences
                    usage_comparison = recommended_usage_pattern.compare(true_usage_pattern)

                    if usage_comparison.empty:
                        accuracy_grid[hh].at[u, a] = 0
                    else:
                        difference = usage_comparison.xs('self', axis=1, level=1, drop_level=False).count().sum()
                        accuracy_grid[hh].at[u, a] = difference

            accuracy_grid[hh] = accuracy_grid[hh].astype(int)

        return accuracy_grid

    def return_best_recommendations(self, accuracy_grid, daily_recommendations_dict):
        threshold_dict = {}

        for hh in self.shiftable_devices_dict.keys():
            min_value = float("inf")
            min_index = None

            for col in accuracy_grid[hh].columns:
                for index, value in enumerate(accuracy_grid[hh][col]):
                    if value < min_value:
                        min_value = value
                        min_index = (index, col)

            usage_threshold = accuracy_grid[hh].index[min_index[0]]
            activity_threshold = min_index[1]

            threshold_dict[hh] = {
                'usage': usage_threshold,
                'activity': activity_threshold
            }

        return threshold_dict

    def pipeline(self):
    
        recommendation_agent_dict = self.initialize_recommendation_agent()
        
        usage_prob_list, activity_prob_list = self.create_threshold_values()
            
        daily_recommendations_dict = self.grid_search_probability_thresholds(usage_prob_list, activity_prob_list, recommendation_agent_dict)
            
        accuracy_grid = self.create_accuracy_grid(usage_prob_list, activity_prob_list, daily_recommendations_dict)
            
        best_thresholds = self.return_best_recommendations(accuracy_grid, daily_recommendations_dict)
            
        return best_thresholds


## **Appendix A2: Complete Create_Accept_Recommendations Class**

In [ ]:
   
class Create_Accept_Recommendations:
    '''
    A class for generating recommendations and simulating device usage patterns through the test phase
    Parameters:
    ----------
    recommendation_agent_dict : 
        A dictionary containing household recommendation agents to create recommendations
    devices :
        A dictionary containing device-specific information
    recommendation_start :
        The start date of the recommendation validation period in 'yyyy-mm-dd' format
    recommendation_length : 
        The length of the recommendation validation period in days
    threshold_dict : 
        A dictionary defining the availability and usage threshold parameter space
    load_dict :
        A dictionary containing the output dataframes from the Load Agent for each household
    '''
    def __init__(
        self,
        recommendation_agent_dict,
        devices: Dict[str, Any],
        recommendation_start: str,
        recommendation_length: int,
        best_thresholds,
        load_dict: Dict[str, pd.DataFrame],
    ):
        import pandas as pd
        from helper_functions_thesis import Helper_Functions_Thesis
        self.recommendation_agent_dict = recommendation_agent_dict
        self.devices = devices
        self.best_thresholds = best_thresholds
        self.recommendation_start = recommendation_start
        self.recommendation_length = recommendation_length
        self.shiftable_devices_dict = Helper_Functions_Thesis.create_shiftable_devices_dict(devices)
        self.date_list = Helper_Functions_Thesis.create_date_list_daily(
            recommendation_start, recommendation_length
        )
        self.load_dict = {dev: df.copy() for dev, df in load_dict.items()}

    def generate_daily_recommendations(self):
        from agents import X_Recommendation_Agent
        from helper_functions_thesis import Helper_Functions_Thesis
        best_daily_recommendations_dict = {}

        for hh in self.shiftable_devices_dict.keys():
            activity_threshold = self.best_thresholds[hh]['activity']
            usage_threshold = self.best_thresholds[hh]['usage']
            print(f"Household: {hh}, Activity Threshold: {activity_threshold}, Usage Threshold: {usage_threshold}")

            best_daily_recommendations_dict[hh] = Helper_Functions_Thesis.create_recommendations(
                date_list=self.date_list,
                hh=hh,
                recommendation_agent_dict=self.recommendation_agent_dict,
                activity_prob_threshold=activity_threshold,
                usage_prob_threshold=usage_threshold
            )
        
        return best_daily_recommendations_dict

    def update_device_information(self, recommendations_dict):
        from agents import Load_Agent
        
        for dev, device_information in self.devices.items():
            hh = device_information['hh']
            dev_name = device_information['dev_name']

            load_agent = Load_Agent(hh[2:])
            load_profiles = load_agent.pipeline(self.load_dict[hh], self.recommendation_start, self.shiftable_devices_dict[hh]) 

            device_information['usage'] = self.load_dict[hh][dev_name]
            device_information['load_profile'] = load_profiles.loc[dev_name]
            device_information['recommendation'] = recommendations_dict[hh][recommendations_dict[hh]['device'] == dev_name]

    def accept_recommendations(self, is_usage_added=False, is_info_displayed=False):
        from datetime import timedelta
        import numpy as np
        import pandas as pd
        
        load_post_recommendation = {}
        
        # Iterate over each device
        for dev in self.devices.keys():
            device_usage = self.devices[dev]['usage'].copy()
            load_profile = self.devices[dev]['load_profile'].copy()
            recommendation = self.devices[dev]['recommendation'].copy()
        
            device_usage = device_usage[str(self.date_list[0]):str(self.date_list[-1])]
            output = device_usage.copy()

            # Iterate over each date in the specified date list
            for date in self.date_list:

                date_before = date - timedelta(days=1)
                date = str(date)

                daily_usage = device_usage.loc[date].copy()
                daily_usage_extended = device_usage.loc[date:][:48].copy()
                
                # Retrieve the recommended start hour
                recommended_start_hour = recommendation[(recommendation.recommendation_date == date)]['recommendation'].iloc[0]
                
                if np.isnan(recommended_start_hour):
                    print(date, "No recommendation") if is_info_displayed else None
                    continue

                recommended_start_hour = int(recommended_start_hour)
                
                # Extract the usage start hours
                shifted_daily_usage = daily_usage.shift()

                if date != str(self.date_list[0]):
                    shifted_daily_usage.iloc[0] = device_usage.loc[str(date_before)].iloc[-1]

                start_hours_usage = daily_usage[
                    (daily_usage.values > 0) & (shifted_daily_usage.values == 0)
                ].index.hour.tolist()

                if start_hours_usage == []:
                    if is_usage_added:
                        print(date, "No usage, Insert average load to the recommended usage hour:", recommended_start_hour) if is_info_displayed else None
                        daily_usage_extended[recommended_start_hour:recommended_start_hour+3] = load_profile[0:3]
                        daily_usage_slice = daily_usage_extended[recommended_start_hour:recommended_start_hour+3]
                        output = daily_usage_slice.combine_first(output)
                        continue
                    else:
                        print(date, "No usage") if is_info_displayed else None
                        continue

                # Determine the closest start hour to the recommended hour
                distance_of_closest_start = int(min(np.full(shape=len(start_hours_usage), fill_value=recommended_start_hour) - start_hours_usage, key=abs, default=0))
                shifted_usage_start = int(recommended_start_hour - distance_of_closest_start)
                
                if (len(start_hours_usage) != 0) and (shifted_usage_start == recommended_start_hour):
                    print(date, "Device usage starts at the recommended hour") if is_info_displayed else None
                    continue

                # Calculate the length of device usage
                usage_length = (
                    ((daily_usage_extended.iloc[shifted_usage_start:] == 0).idxmax() - daily_usage_extended.index[shifted_usage_start])
                        // pd.Timedelta(minutes=60))

                print(date, "Shift load starting at", shifted_usage_start, "to the recommended hour", recommended_start_hour) if is_info_displayed else None

                usage_section = daily_usage_extended.iloc[shifted_usage_start:shifted_usage_start + usage_length]
                usage_shifted = usage_section.shift(periods=distance_of_closest_start, freq="60T")
                output[date:].iloc[shifted_usage_start:shifted_usage_start + usage_length] = 0
                output = usage_shifted.combine_first(output)

            # Store the results for the current device
            output = pd.DataFrame({'usage': output})
            load_post_recommendation[dev] = output
        
        return load_post_recommendation

    def pipeline(self):
        recommendations_test_dict = self.generate_daily_recommendations()

        self.update_device_information(recommendations_test_dict)
        
        load_post_recommendation = self.accept_recommendations(is_usage_added=False, is_info_displayed=False)
        
        load_post_recommendation_usage_added = self.accept_recommendations(is_usage_added=True, is_info_displayed=False)
        
        return load_post_recommendation, load_post_recommendation_usage_added